# Baseline Model 1: 1D-CNN (Spilka et al., 2016)

**Objective:** Reproduce the Deep Learning baseline using a standard 1D-CNN architecture on FHR signals ONLY (Unimodal).

**Reference:** Spilka, J., et al. (2016). *Deep Learning for Fetal Heart Rate Analysis.*

## 1. Setup & Imports

In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import matplotlib.pyplot as plt

# --- GitHub & Colab Setup ---
try:
    from google.colab import userdata

    # 1. Clone Repo using Secret Token
    token = userdata.get('GITHUB_AUTH_TOKEN')
    repo_name = "NeuroFetal-AI"
    username = "Krishna200608"
    repo_url = f"https://{token}@github.com/{username}/{repo_name}.git"

    if not os.path.exists(repo_name):
        print(f"Cloning {repo_name}...")
        get_ipython().system(f"git clone {repo_url}")

    # 2. Configure Git
    os.chdir(repo_name)
    get_ipython().system('git config --global user.email "krishnasikheriya001@gmail.com"')
    get_ipython().system('git config --global user.name "Krishna200608"')

    # 3. Install Dependencies
    get_ipython().system('pip install wfdb')

    BASE_DIR = os.getcwd()
    sys.path.append(os.path.join(BASE_DIR, "Code", "scripts"))
    print("Running in Colab (GitHub Integration Active)")

except ImportError:
    # Local Fallback
    BASE_DIR = os.path.abspath(os.path.join("..", ".."))
    sys.path.append(os.path.abspath(os.path.join("..", "scripts")))
    print("Running Locally")

import data_ingestion
print(f"TensorFlow Version: {tf.__version__}")

Running in Colab (GitHub Integration Active)
TensorFlow Version: 2.19.0


## 2. Load and Process Data
Using the existing `data_ingestion` pipeline. If data is missing locally, we attempt to run the ingestion script.

In [2]:
# Parameters
WINDOW_SIZE = 2400  # 20 minutes at 2Hz
STRIDE = 300

PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "Datasets", "processed")
X_path = os.path.join(PROCESSED_DATA_DIR, "X_fhr.npy")
y_path = os.path.join(PROCESSED_DATA_DIR, "y.npy")

# Check if data exists, if not, run ingestion
if not os.path.exists(X_path) or not os.path.exists(y_path):
    print("Processed data not found. Running data_ingestion.py...")
    # This assumes data_ingestion.py handles download/processing
    get_ipython().system(f"python Code/scripts/data_ingestion.py")

try:
    X_signal = np.load(X_path)
    y = np.load(y_path)
    print(f"Loaded Data: X_signal {X_signal.shape}, y {y.shape}")
except FileNotFoundError:
    print("Error: Data ingestion failed or data not found.")
    # Stop execution if critical
    raise

Loaded Data: X_signal (2546, 1200), y (2546,)


## 3. Define Baseline CNN (Spilka et al.)

In [3]:
def build_baseline_cnn(input_shape=(2400, 1)):
    inputs = keras.Input(shape=input_shape)

    # Block 1
    x = layers.Conv1D(filters=16, kernel_size=7, strides=1, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # Block 2
    x = layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # Block 3
    x = layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling1D()(x)

    # Dense Classification Head
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="Baseline_CNN")
    return model

model = build_baseline_cnn(input_shape=(X_signal.shape[1], 1))
model.summary()

Model: "Baseline_CNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1200, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1200, 16)       │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1200, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 1200, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 600, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 600, 32)        │         2,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 600, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 600, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 300, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 300, 64)        │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 300, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 300, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,601 (53.13 KB)

 Trainable params: 13,377 (52.25 KB)

 Non-trainable params: 224 (896.00 B)

## 4. Training (5-Fold CV)

In [4]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

aucs = []
accs = []

X = X_signal
if X.ndim == 2:
    X = np.expand_dims(X, axis=-1)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print(f"\nStarting Fold {fold+1}...")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Class weights
    y_train = y_train.astype(int)
    neg, pos = np.bincount(y_train)
    class_weight = {0: 1.0, 1: (neg / pos) if pos > 0 else 1.0}

    model = build_baseline_cnn(input_shape=(X.shape[1], 1))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC', 'accuracy'])

    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_auc', patience=10, mode='max', restore_best_weights=True
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        class_weight=class_weight,
        callbacks=[early_stopping],
        verbose=1
    )

    y_pred = model.predict(X_val, verbose=0)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))

    aucs.append(auc)
    accs.append(acc)
    print(f"Fold {fold+1} Result -> AUC: {auc:.4f}, Acc: {acc:.4f}")

    # Save Best Model
    if len(aucs) == 1 or auc > max(aucs[:-1]):
        model_dir = os.path.join(BASE_DIR, "Code", "Baseline", "Models")
        os.makedirs(model_dir, exist_ok=True)
        model_save_path = os.path.join(model_dir, "baseline_paper3_best_cnn.keras")
        model.save(model_save_path)
        print(f"  Saved best model to {model_save_path}")

print("\n=== Final Results ===")
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)
mean_acc = np.mean(accs)
print(f"Mean AUC: {mean_auc:.4f} +/- {std_auc:.4f}")
print(f"Mean Acc: {mean_acc:.4f}")


Starting Fold 1...
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 84ms/step - AUC: 0.4962 - accuracy: 0.5138 - loss: 1.1720 - val_AUC: 0.4884 - val_accuracy: 0.8157 - val_loss: 0.6737
Epoch 2/50
39/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.5545 - accuracy: 0.4787 - loss: 1.1333

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,accuracy,loss,val_AUC,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5488 - accuracy: 0.5077 - loss: 1.1310 - val_AUC: 0.4752 - val_accuracy: 0.8157 - val_loss: 0.6474
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5411 - accuracy: 0.4626 - loss: 1.1464 - val_AUC: 0.5279 - val_accuracy: 0.8157 - val_loss: 0.6234
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5851 - accuracy: 0.6488 - loss: 1.1164 - val_AUC: 0.5091 - val_accuracy: 0.8157 - val_loss: 0.6264
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5272 - accuracy: 0.5323 - loss: 1.1219 - val_AUC: 0.5182 - val_accuracy: 0.8157 - val_loss: 0.6427
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5900 - accuracy: 0.5958 - loss: 1.0691 - val_AUC: 0.5417 - val_accuracy: 0.7706 - val_loss: 0.6630
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5638 - accuracy: 0.4497 - loss: 1.1086 - val_AUC: 0.5390 - val_accuracy: 0.7902 - val_loss: 0.6553
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5326

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,accuracy,loss,val_AUC,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5283 - accuracy: 0.5224 - loss: 1.1108 - val_AUC: 0.5495 - val_accuracy: 0.8153 - val_loss: 0.6544
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5438 - accuracy: 0.4946 - loss: 1.1570 - val_AUC: 0.5348 - val_accuracy: 0.8153 - val_loss: 0.6022
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5941 - accuracy: 0.6801 - loss: 1.0888 - val_AUC: 0.5246 - val_accuracy: 0.8153 - val_loss: 0.6412
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5489 - accuracy: 0.5838 - loss: 1.0932 - val_AUC: 0.5513 - val_accuracy: 0.8153 - val_loss: 0.6368
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5942 - accuracy: 0.5498 - loss: 1.1015 - val_AUC: 0.5410 - val_accuracy: 0.8153 - val_loss: 0.6311
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5671 - accuracy: 0.6048 - loss: 1.1085 - val_AUC: 0.5445 - val_accuracy: 0.8114 - val_loss: 0.6407
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5685

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5294 - accuracy: 0.4816 - loss: 1.1611 - val_AUC: 0.5167 - val_accuracy: 0.8153 - val_loss: 0.6238
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5708 - accuracy: 0.6050 - loss: 1.1260 - val_AUC: 0.5305 - val_accuracy: 0.8153 - val_loss: 0.6146
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5618 - accuracy: 0.6003 - loss: 1.1174 - val_AUC: 0.5180 - val_accuracy: 0.7564 - val_loss: 0.6470
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5876 - accuracy: 0.5014 - loss: 1.1013 - val_AUC: 0.5245 - val_accuracy: 0.7976 - val_loss: 0.6392
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - AUC: 0.5571 - accuracy: 0.5295 - loss: 1.1336 - val_AUC: 0.5339 - val_accuracy: 0.6935 - val_loss: 0.6756
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - AUC: 0.5562 - accuracy: 0.5327 - loss: 1.1293 - val_AUC: 0.5289 - val_accuracy: 0.7819 - val_loss: 0.6329
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - AUC: 0.5702

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,accuracy,loss,val_AUC,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5056 - accuracy: 0.4499 - loss: 1.1453 - val_AUC: 0.4666 - val_accuracy: 0.8153 - val_loss: 0.6362
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5101 - accuracy: 0.5099 - loss: 1.1269 - val_AUC: 0.5640 - val_accuracy: 0.8153 - val_loss: 0.6355
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5679 - accuracy: 0.5801 - loss: 1.0993 - val_AUC: 0.5127 - val_accuracy: 0.8153 - val_loss: 0.6654
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5687 - accuracy: 0.4955 - loss: 1.1009 - val_AUC: 0.5493 - val_accuracy: 0.5619 - val_loss: 0.6906
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5947 - accuracy: 0.5368 - loss: 1.1305 - val_AUC: 0.5768 - val_accuracy: 0.4538 - val_loss: 0.6987
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5911 - accuracy: 0.5375 - loss: 1.1253 - val_AUC: 0.5317 - val_accuracy: 0.6483 - val_loss: 0.6778
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5464

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: AUC,accuracy,loss,val_AUC,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - AUC: 0.5041 - accuracy: 0.5506 - loss: 1.1513 - val_AUC: 0.5322 - val_accuracy: 0.8153 - val_loss: 0.6447
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5043 - accuracy: 0.5487 - loss: 1.1833 - val_AUC: 0.5535 - val_accuracy: 0.8153 - val_loss: 0.6232
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5262 - accuracy: 0.5606 - loss: 1.1323 - val_AUC: 0.5521 - val_accuracy: 0.8153 - val_loss: 0.6151
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - AUC: 0.5241 - accuracy: 0.5596 - loss: 1.0888 - val_AUC: 0.5565 - val_accuracy: 0.8153 - val_loss: 0.6210
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5540 - accuracy: 0.5858 - loss: 1.0965 - val_AUC: 0.5887 - val_accuracy: 0.8114 - val_loss: 0.6283
Epoch 7/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5606 - accuracy: 0.5929 - loss: 1.0855 - val_AUC: 0.5681 - val_accuracy: 0.7898 - val_loss: 0.6481
Epoch 8/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.5716

In [5]:
# 5. Save Results & Push to GitHub

results_path = os.path.join(BASE_DIR, "Code", "Baseline", "baseline_paper3_results.txt")
with open(results_path, "w") as f:
    f.write(f"Ref: Paper 3 (Spilka 2016) - CNN Baseline (Colab Run)\n")
    f.write(f"Mean AUC: {mean_auc:.4f}\n")
    f.write(f"Std Dev: {std_auc:.4f}\n")
    f.write(f"Mean Acc: {mean_acc:.4f}\n")
print(f"Results saved to {results_path}")

# Git Commit & Push
try:
    if 'google.colab' in sys.modules:
        print("Pushing results to GitHub...")
        # Ensure we are in repo root
        os.chdir(BASE_DIR)

        # Configure again just in case
        get_ipython().system('git config --global user.email "krishnasikheriya001@gmail.com"')
        get_ipython().system('git config --global user.name "Krishna200608"')

        # Pull latest changes to avoid conflicts
        get_ipython().system('git pull origin main')

        # Add relevant files
        get_ipython().system('git add Code/Baseline/Models/*.keras')
        get_ipython().system('git add Code/Baseline/*.txt')

        # Commit
        get_ipython().system('git commit -m "Update CNN Baseline Results (Colab)"')

        # Push
        get_ipython().system('git push origin main')
        print("Successfully pushed to GitHub!")
except Exception as e:
    print(f"Git Push Failed: {e}")

Results saved to /content/NeuroFetal-AI/Code/Baseline/baseline_paper3_results.txt
Pushing results to GitHub...
From https://github.com/Krishna200608/NeuroFetal-AI
 * branch            main       -> FETCH_HEAD
Already up to date.
[main ed00edf] Update CNN Baseline Results (Colab)
 2 files changed, 4 insertions(+), 4 deletions(-)
 create mode 100644 Code/Baseline/Models/baseline_paper3_best_cnn.keras
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 152.50 KiB | 13.86 MiB/s, done.
Total 7 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/Krishna200608/NeuroFetal-AI.git
   35c86e0..ed00edf  main -> main
Successfully pushed to GitHub!
